In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)


In [2]:
class AnalisadorAcidentesPBIC:
    def __init__(self, pasta_dados='./'):
        """
        Inicializa o analisador de acidentes PBIC
        Args:
            pasta_dados (str): Caminho para a pasta com os arquivos CSV
        """
        self.pasta_dados = Path(pasta_dados)
        self.df = None
        self.df_limpo = None
        self.encoders = {}
        
    def carregar_dados(self):
        """Carrega e combina todos os arquivos CSV de 2020-2025"""
        print("🔄 Carregando dados de acidentes (2020-2025)...")
        
        arquivos_csv = list(self.pasta_dados.glob("acidentes*_todas_causas_tipos.csv"))
        arquivos_csv.sort()
        
        if not arquivos_csv:
            raise FileNotFoundError("Nenhum arquivo de acidentes encontrado!")
        
        dataframes = []
        for arquivo in arquivos_csv:
            print(f"   📁 Carregando {arquivo.name}...")
            
            encodings = ['utf-8', 'latin1', 'iso-8859-1', 'cp1252']
            df_temp = None
            
            for encoding in encodings:
                try:
                    df_temp = pd.read_csv(arquivo, sep=';', encoding=encoding, low_memory=False)
                    print(f"      ✅ Encoding {encoding} funcionou")
                    break
                except UnicodeDecodeError:
                    continue
            
            if df_temp is None:
                raise ValueError(f"Não foi possível ler o arquivo {arquivo.name} com nenhum encoding testado")
            
            ano = arquivo.name.split('acidentes')[1][:4]
            df_temp['ano_arquivo'] = int(ano)
            
            dataframes.append(df_temp)
            print(f"      ✅ {len(df_temp):,} registros carregados")
        
        self.df = pd.concat(dataframes, ignore_index=True)
        print(f"\n✅ Dados combinados: {len(self.df):,} registros totais")
        print(f"📊 Período: {self.df['ano_arquivo'].min()} - {self.df['ano_arquivo'].max()}")
        
        return self.df
    
    def explorar_dados_basicos(self):
        """Análise exploratória básica dos dados"""
        print("\n" + "="*60)
        print("📊 ANÁLISE EXPLORATÓRIA BÁSICA")
        print("="*60)
        
        print(f"📈 Dimensões do dataset: {self.df.shape}")
        print(f"📅 Período dos dados: {self.df['ano_arquivo'].min()} - {self.df['ano_arquivo'].max()}")
        
        print(f"\n🔍 Informações das colunas:")
        print(f"   • Total de colunas: {len(self.df.columns)}")
        print(f"   • Valores ausentes: {self.df.isnull().sum().sum():,}")
        
        print(f"\n📊 Distribuição de acidentes por ano:")
        dist_ano = self.df['ano_arquivo'].value_counts().sort_index()
        for ano, qtd in dist_ano.items():
            print(f"   • {ano}: {qtd:,} acidentes")
        
        print(f"\n🚑 Estatísticas de vítimas:")
        for col in ['ilesos', 'feridos_leves', 'feridos_graves', 'mortos']:
            if col in self.df.columns:
                total = self.df[col].sum()
                print(f"   • {col.replace('_', ' ').title()}: {total:,}")
        
        return self.df.describe()
    
    def limpar_dados(self):
        """Limpeza e preparação dos dados"""
        print("\n" + "="*60)
        print("🧹 LIMPEZA E PREPARAÇÃO DOS DADOS")
        print("="*60)
        
        self.df_limpo = self.df.copy()
        
        if 'data_inversa' in self.df_limpo.columns:
            self.df_limpo['data_inversa'] = pd.to_datetime(self.df_limpo['data_inversa'], errors='coerce')
            self.df_limpo['mes'] = self.df_limpo['data_inversa'].dt.month
            self.df_limpo['dia_mes'] = self.df_limpo['data_inversa'].dt.day
            print("✅ Data convertida e features temporais criadas")
        
        if 'horario' in self.df_limpo.columns:
            self.df_limpo['hora'] = pd.to_datetime(self.df_limpo['horario'], format='%H:%M:%S', errors='coerce').dt.hour
            print("✅ Horário convertido")
        
        self.df_limpo['gravidade_numerica'] = 0
        if 'feridos_leves' in self.df_limpo.columns:
            self.df_limpo.loc[self.df_limpo['feridos_leves'] > 0, 'gravidade_numerica'] = 1
        if 'feridos_graves' in self.df_limpo.columns:
            self.df_limpo.loc[self.df_limpo['feridos_graves'] > 0, 'gravidade_numerica'] = 2
        if 'mortos' in self.df_limpo.columns:
            self.df_limpo.loc[self.df_limpo['mortos'] > 0, 'gravidade_numerica'] = 3
        
        colunas_vitimas = ['ilesos', 'feridos_leves', 'feridos_graves', 'mortos']
        colunas_existentes = [col for col in colunas_vitimas if col in self.df_limpo.columns]
        if colunas_existentes:
            self.df_limpo['total_vitimas'] = self.df_limpo[colunas_existentes].sum(axis=1)
        
        antes = len(self.df_limpo)
        self.df_limpo = self.df_limpo.dropna(subset=['uf', 'municipio'])
        depois = len(self.df_limpo)
        
        print(f"✅ Limpeza concluída: {antes - depois:,} registros removidos")
        print(f"📊 Dataset limpo: {depois:,} registros")
        
        return self.df_limpo
    
    def salvar_dados_limpos(self):
        """Salva o dataset limpo em arquivo CSV"""
        print("\n" + "="*60)
        print("💾 SALVANDO DADOS LIMPOS")
        print("="*60)
        
        nome_arquivo = 'acidentes_pbic_2020_2025_limpo.csv'
        self.df_limpo.to_csv(nome_arquivo, index=False, encoding='utf-8')
        
        print(f"✅ Dataset limpo salvo como: {nome_arquivo}")
        print(f"📊 Total de registros salvos: {len(self.df_limpo):,}")
        print(f"📊 Total de colunas: {len(self.df_limpo.columns)}")
        
        return nome_arquivo
    
    def relatorio_final(self):
        """Gera relatório final do processamento"""
        print("\n" + "="*60)
        print("📋 RELATÓRIO FINAL")
        print("="*60)
        
        print(f"✅ Processamento concluído com sucesso!")
        print(f"📊 Total de registros processados: {len(self.df_limpo):,}")
        print(f"📅 Período: {self.df_limpo['ano_arquivo'].min()} - {self.df_limpo['ano_arquivo'].max()}")
        
        if 'total_vitimas' in self.df_limpo.columns:
            total_vitimas = self.df_limpo['total_vitimas'].sum()
            print(f"🚑 Total de vítimas no período: {total_vitimas:,}")
        
        print(f"\n📁 Arquivo gerado:")
        print(f"   • acidentes_pbic_2020_2025_limpo.csv - Dataset limpo e combinado")
        
        print(f"\n💡 Próximos passos sugeridos:")
        print(f"   • Usar o arquivo limpo para análises específicas")
        print(f"   • Consultar sugestoes_analises_pbic.md para ideias de análises")
        print(f"   • Implementar visualizações e modelos de ML")
        
    def contar_dimensoes_por_ano(self):
        """
        Imprime o número de linhas e colunas de cada ano separadamente,
        e depois mostra o total combinado de linhas e colunas.
        """
        if self.df is None:
            raise ValueError("Nenhum dado carregado. Execute carregar_dados() primeiro.")
        
        print("\n" + "="*60)
        print("📊 Contagem de linhas e colunas por ano")
        print("="*60)
        
        total_linhas = 0
        total_colunas = 0
        
        for ano in sorted(self.df['ano_arquivo'].unique()):
            df_ano = self.df[self.df['ano_arquivo'] == ano]
            linhas = len(df_ano)
            colunas = len(df_ano.columns)
            total_linhas += linhas
            total_colunas += colunas  # aqui somamos colunas de cada ano
            
            print(f"📅 Ano {ano}: {linhas:,} linhas, {colunas:,} colunas")
        
        print("\n" + "-"*60)
        print(f"✅ Total combinado: {total_linhas:,} linhas, {total_colunas:,} colunas")
        print("-"*60)                           

In [3]:
# Inicializa o analisador
analisador = AnalisadorAcidentesPBIC('./DadosPRF')

# 1. Carregar dados
df = analisador.carregar_dados()
df.head()


🔄 Carregando dados de acidentes (2020-2025)...
   📁 Carregando acidentes2020_todas_causas_tipos.csv...
      ✅ Encoding latin1 funcionou
      ✅ 384,640 registros carregados
   📁 Carregando acidentes2021_todas_causas_tipos.csv...
      ✅ Encoding latin1 funcionou
      ✅ 436,523 registros carregados
   📁 Carregando acidentes2022_todas_causas_tipos.csv...
      ✅ Encoding latin1 funcionou
      ✅ 507,204 registros carregados
   📁 Carregando acidentes2023_todas_causas_tipos.csv...
      ✅ Encoding latin1 funcionou
      ✅ 571,052 registros carregados
   📁 Carregando acidentes2024_todas_causas_tipos.csv...
      ✅ Encoding latin1 funcionou
      ✅ 603,215 registros carregados
   📁 Carregando acidentes2025_todas_causas_tipos.csv...
      ✅ Encoding latin1 funcionou
      ✅ 279,907 registros carregados

✅ Dados combinados: 2,782,541 registros totais
📊 Período: 2020 - 2025


,id,pesid,data_inversa,dia_semana,horario,uf,br,km,municipio,causa_principal,causa_acidente,ordem_tipo_acidente,tipo_acidente,classificacao_acidente,fase_dia,sentido_via,condicao_metereologica,tipo_pista,tracado_via,uso_solo,id_veiculo,tipo_veiculo,marca,ano_fabricacao_veiculo,tipo_envolvido,estado_fisico,idade,sexo,ilesos,feridos_leves,feridos_graves,mortos,latitude,longitude,regional,delegacia,uop,ano_arquivo
0,260031.0,578988.0,2020-01-01,quarta-feira,01:00:00,TO,153.0,"678,1",GURUPI,Sim,Animais na Pista,2,Saída de leito carroçável,Com Vítimas Feridas,Plena Noite,Crescente,Nublado,Simples,Reta,Não,464101.0,Automóvel,FIAT/SIENA ATTRACTIV 1.4/SIENA ATTRACTIV 1.4,2012.0,Passageiro,Lesões Graves,18.0,Masculino,0.0,0.0,1.0,0.0,"-11,77460203","-49,10744996",SPRF-TO,DEL02-TO,UOP02-DEL01-TO,2020
1,260031.0,578987.0,2020-01-01,quarta-feira,01:00:00,TO,153.0,"678,1",GURUPI,Sim,Animais na Pista,2,Saída de leito carroçável,Com Vítimas Feridas,Plena Noite,Crescente,Nublado,Simples,Reta,Não,464101.0,Automóvel,FIAT/SIENA ATTRACTIV 1.4/SIENA ATTRACTIV 1.4,2012.0,Passageiro,Lesões Graves,20.0,Masculino,0.0,0.0,1.0,0.0,"-11,77460203","-49,10744996",SPRF-TO,DEL02-TO,UOP02-DEL01-TO,2020
2,260031.0,578991.0,2020-01-01,quarta-feira,01:00:00,TO,153.0,"678,1",GURUPI,Sim,Animais na Pista,2,Saída de leito carroçável,Com Vítimas Feridas,Plena Noite,Crescente,Nublado,Simples,Reta,Não,464101.0,Automóvel,FIAT/SIENA ATTRACTIV 1.4/SIENA ATTRACTIV 1.4,2012.0,Passageiro,Lesões Graves,17.0,Feminino,0.0,0.0,1.0,0.0,"-11,77460203","-49,10744996",SPRF-TO,DEL02-TO,UOP02-DEL01-TO,2020
3,260031.0,578986.0,2020-01-01,quarta-feira,01:00:00,TO,153.0,"678,1",GURUPI,Sim,Animais na Pista,2,Saída de leito carroçável,Com Vítimas Feridas,Plena Noite,Crescente,Nublado,Simples,Reta,Não,464101.0,Automóvel,FIAT/SIENA ATTRACTIV 1.4/SIENA ATTRACTIV 1.4,2012.0,Passageiro,Lesões Graves,16.0,Feminino,0.0,0.0,1.0,0.0,"-11,77460203","-49,10744996",SPRF-TO,DEL02-TO,UOP02-DEL01-TO,2020
4,260031.0,578475.0,2020-01-01,quarta-feira,01:00:00,TO,153.0,"678,1",GURUPI,Sim,Animais na Pista,2,Saída de leito carroçável,Com Vítimas Feridas,Plena Noite,Crescente,Nublado,Simples,Reta,Não,464101.0,Automóvel,FIAT/SIENA ATTRACTIV 1.4/SIENA ATTRACTIV 1.4,2012.0,Condutor,Lesões Graves,33.0,Feminino,0.0,0.0,1.0,0.0,"-11,77460203","-49,10744996",SPRF-TO,DEL02-TO,UOP02-DEL01-TO,2020


In [4]:
# 2. Análise exploratória básica
analisador.explorar_dados_basicos()



📊 ANÁLISE EXPLORATÓRIA BÁSICA
📈 Dimensões do dataset: (2782541, 38)
📅 Período dos dados: 2020 - 2025

🔍 Informações das colunas:
   • Total de colunas: 38
   • Valores ausentes: 3,011,605

📊 Distribuição de acidentes por ano:
   • 2020: 384,640 acidentes
   • 2021: 436,523 acidentes
   • 2022: 507,204 acidentes
   • 2023: 571,052 acidentes
   • 2024: 603,215 acidentes
   • 2025: 279,907 acidentes

🚑 Estatísticas de vítimas:
   • Ilesos: 1,021,198.0
   • Feridos Leves: 865,058.0
   • Feridos Graves: 320,351.0
   • Mortos: 131,563.0


,id,pesid,br,ordem_tipo_acidente,id_veiculo,ano_fabricacao_veiculo,idade,ilesos,feridos_leves,feridos_graves,mortos,ano_arquivo
count,2.782541e+06,2.510785e+06,2.774532e+06,2.782541e+06,2.683028e+06,2.683028e+06,2.265150e+06,2.510785e+06,2.510785e+06,2.510785e+06,2.510785e+06,2.782541e+06
mean,4.941932e+05,1.096391e+06,2.161420e+02,1.712527e+00,8.813602e+05,1.905128e+03,3.953279e+01,4.067246e-01,3.445369e-01,1.275900e-01,5.239915e-02,2.022507e+03
std,1.234481e+05,2.776550e+05,1.276409e+02,9.986492e-01,2.200486e+05,4.515307e+02,4.025365e+01,4.912228e-01,4.752171e-01,3.336328e-01,2.228307e-01,1.558924e+00
min,2.600310e+05,5.784610e+05,1.000000e+01,1.000000e+00,4.640970e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.020000e+03
25%,3.870160e+05,8.645860e+05,1.160000e+02,1.000000e+00,7.008040e+05,2.007000e+03,2.700000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.021000e+03
50%,5.061410e+05,1.107283e+06,1.630000e+02,1.000000e+00,8.920595e+05,2.012000e+03,3.700000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.023000e+03
75%,5.986960e+05,1.335107e+06,3.430000e+02,2.000000e+00,1.069945e+06,2.018000e+03,4.900000e+01,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,2.024000e+03
max,7.058830e+05,1.571987e+06,4.980000e+02,1.500000e+01,1.254449e+06,2.025000e+03,2.024000e+03,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,2.025000e+03


In [5]:
# Contagem de linhas e colunas por ano e total
analisador.contar_dimensoes_por_ano()



📊 Contagem de linhas e colunas por ano
📅 Ano 2020: 384,640 linhas, 38 colunas
📅 Ano 2021: 436,523 linhas, 38 colunas
📅 Ano 2022: 507,204 linhas, 38 colunas
📅 Ano 2023: 571,052 linhas, 38 colunas
📅 Ano 2024: 603,215 linhas, 38 colunas
📅 Ano 2025: 279,907 linhas, 38 colunas

------------------------------------------------------------
✅ Total combinado: 2,782,541 linhas, 228 colunas
------------------------------------------------------------


In [6]:
# 3. Limpeza dos dados
df_limpo = analisador.limpar_dados()
df_limpo.head()



🧹 LIMPEZA E PREPARAÇÃO DOS DADOS
✅ Data convertida e features temporais criadas
✅ Horário convertido
✅ Limpeza concluída: 0 registros removidos
📊 Dataset limpo: 2,782,541 registros


,id,pesid,data_inversa,dia_semana,horario,uf,br,km,municipio,causa_principal,causa_acidente,ordem_tipo_acidente,tipo_acidente,classificacao_acidente,fase_dia,sentido_via,condicao_metereologica,tipo_pista,tracado_via,uso_solo,id_veiculo,tipo_veiculo,marca,ano_fabricacao_veiculo,tipo_envolvido,estado_fisico,idade,sexo,ilesos,feridos_leves,feridos_graves,mortos,latitude,longitude,regional,delegacia,uop,ano_arquivo,mes,dia_mes,hora,gravidade_numerica,total_vitimas
0,260031.0,578988.0,2020-01-01,quarta-feira,01:00:00,TO,153.0,"678,1",GURUPI,Sim,Animais na Pista,2,Saída de leito carroçável,Com Vítimas Feridas,Plena Noite,Crescente,Nublado,Simples,Reta,Não,464101.0,Automóvel,FIAT/SIENA ATTRACTIV 1.4/SIENA ATTRACTIV 1.4,2012.0,Passageiro,Lesões Graves,18.0,Masculino,0.0,0.0,1.0,0.0,"-11,77460203","-49,10744996",SPRF-TO,DEL02-TO,UOP02-DEL01-TO,2020,1,1,1,2,1.0
1,260031.0,578987.0,2020-01-01,quarta-feira,01:00:00,TO,153.0,"678,1",GURUPI,Sim,Animais na Pista,2,Saída de leito carroçável,Com Vítimas Feridas,Plena Noite,Crescente,Nublado,Simples,Reta,Não,464101.0,Automóvel,FIAT/SIENA ATTRACTIV 1.4/SIENA ATTRACTIV 1.4,2012.0,Passageiro,Lesões Graves,20.0,Masculino,0.0,0.0,1.0,0.0,"-11,77460203","-49,10744996",SPRF-TO,DEL02-TO,UOP02-DEL01-TO,2020,1,1,1,2,1.0
2,260031.0,578991.0,2020-01-01,quarta-feira,01:00:00,TO,153.0,"678,1",GURUPI,Sim,Animais na Pista,2,Saída de leito carroçável,Com Vítimas Feridas,Plena Noite,Crescente,Nublado,Simples,Reta,Não,464101.0,Automóvel,FIAT/SIENA ATTRACTIV 1.4/SIENA ATTRACTIV 1.4,2012.0,Passageiro,Lesões Graves,17.0,Feminino,0.0,0.0,1.0,0.0,"-11,77460203","-49,10744996",SPRF-TO,DEL02-TO,UOP02-DEL01-TO,2020,1,1,1,2,1.0
3,260031.0,578986.0,2020-01-01,quarta-feira,01:00:00,TO,153.0,"678,1",GURUPI,Sim,Animais na Pista,2,Saída de leito carroçável,Com Vítimas Feridas,Plena Noite,Crescente,Nublado,Simples,Reta,Não,464101.0,Automóvel,FIAT/SIENA ATTRACTIV 1.4/SIENA ATTRACTIV 1.4,2012.0,Passageiro,Lesões Graves,16.0,Feminino,0.0,0.0,1.0,0.0,"-11,77460203","-49,10744996",SPRF-TO,DEL02-TO,UOP02-DEL01-TO,2020,1,1,1,2,1.0
4,260031.0,578475.0,2020-01-01,quarta-feira,01:00:00,TO,153.0,"678,1",GURUPI,Sim,Animais na Pista,2,Saída de leito carroçável,Com Vítimas Feridas,Plena Noite,Crescente,Nublado,Simples,Reta,Não,464101.0,Automóvel,FIAT/SIENA ATTRACTIV 1.4/SIENA ATTRACTIV 1.4,2012.0,Condutor,Lesões Graves,33.0,Feminino,0.0,0.0,1.0,0.0,"-11,77460203","-49,10744996",SPRF-TO,DEL02-TO,UOP02-DEL01-TO,2020,1,1,1,2,1.0


In [7]:
# 5. Relatório final
analisador.relatorio_final()



📋 RELATÓRIO FINAL
✅ Processamento concluído com sucesso!
📊 Total de registros processados: 2,782,541
📅 Período: 2020 - 2025
🚑 Total de vítimas no período: 2,338,170.0

📁 Arquivo gerado:
   • acidentes_pbic_2020_2025_limpo.csv - Dataset limpo e combinado

💡 Próximos passos sugeridos:
   • Usar o arquivo limpo para análises específicas
   • Consultar sugestoes_analises_pbic.md para ideias de análises
   • Implementar visualizações e modelos de ML


In [1]:
# Substitua 'nome_do_seu_arquivo.csv' pelo nome do arquivo que você carregou
df = pd.read_csv('acidentes_pbic_2020_2025_limpo.csv')

# Para visualizar as primeiras linhas do DataFrame
display(df.head())

NameError: name 'pd' is not defined

In [4]:
# Passo 2: Verificar número de linhas e colunas
num_linhas, num_colunas = df.shape
print(f"Número de linhas: {num_linhas}")
print(f"Número de colunas: {num_colunas}\n")

Número de linhas: 2782541
Número de colunas: 43



In [5]:
# Passo 3: Verificar colunas duplicadas
colunas_duplicadas = df.columns[df.columns.duplicated()].tolist()
if colunas_duplicadas:
    print(f"Colunas duplicadas: {colunas_duplicadas}")
else:
    print("Não há colunas duplicadas.")

Não há colunas duplicadas.


In [6]:

# Passo 4: Verificar linhas duplicadas
linhas_duplicadas = df[df.duplicated()]
print(f"Número de linhas duplicadas: {linhas_duplicadas.shape[0]}\n")

# Passo 5: Verificar valores nulos

Número de linhas duplicadas: 0



In [7]:
# Passo 5: Verificar valores nulos
valores_nulos = df.isnull().sum()
print("Valores nulos por coluna:")
print(valores_nulos[valores_nulos > 0], "\n")

Valores nulos por coluna:
pesid                     271756
br                          8009
km                          8009
tipo_acidente                  1
classificacao_acidente        86
id_veiculo                 99513
tipo_veiculo               99513
ano_fabricacao_veiculo     99513
tipo_envolvido            271756
estado_fisico             271756
idade                     517391
sexo                      271756
ilesos                    271756
feridos_leves             271756
feridos_graves            271756
mortos                    271756
regional                     369
delegacia                   1430
uop                         3723
dtype: int64 

